In [ ]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# loading csv file into pandas dataframe

# specify path
my_path = "/content/drive/MyDrive/ratings (1).dat"


In [ ]:
# ratings: user_id, item_id, rating
# Read the data from the file
data = []
with open(my_path, 'r') as file:
    for line in file:
        # Split the line by the delimiter '::' and extract UserID, MovieID, and Rating
        parts = line.strip().split(",")
        user_id = int(parts[0])
        movie_id = int(parts[1])
        rating = int(parts[2])

        data.append((user_id, movie_id, rating))


In [ ]:
ratings_matrix= pd.DataFrame(data, columns=['user_id', 'movie_id', 'ratings'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ratings_matrix.head()

,user_id,movie_id,ratings
0,1,1,5
1,1,48,3
2,1,150,3
3,1,260,4
4,1,527,5


In [ ]:
ratings_matrix.tail()


,user_id,movie_id,ratings
1000204,6040,3683,1
1000205,6040,3703,5
1000206,6040,3735,5
1000207,6040,3751,4
1000208,6040,3819,4


In [ ]:
ratings_matrix.shape


(1000209, 3)

In [ ]:
ratings_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   user_id   1000209 non-null  int64
 1   movie_id  1000209 non-null  int64
 2   ratings   1000209 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(ratings_matrix, test_size = 0.30, random_state = 42)

print(X_train.shape)
print(X_test.shape)

(700146, 3)
(300063, 3)


In [ ]:
item_user_matrix = X_train.pivot(index = 'user_id', columns = 'movie_id', values = 'ratings').fillna(0)
item_user_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cosine_similarity_matrix = np.dot(item_user_matrix.T, item_user_matrix) / (np.linalg.norm(item_user_matrix, axis=0)[:, None] * np.linalg.norm(item_user_matrix, axis=0))


In [ ]:
pearson_similarity_matrix = item_user_matrix.corr(method='pearson')

In [ ]:
# Item Similarity Matrix using Cosine similarity as a similarity measure between Items
item_similarity = pearson_similarity_matrix
item_similarity[np.isnan(item_similarity)] = 0
print(item_similarity)
print("- "*10)
print(item_similarity.shape)

movie_id      1         2         3         4         5         6     \
movie_id                                                               
1         1.000000  0.171187  0.084238  0.066051  0.105487  0.082744   
2         0.171187  1.000000  0.132769  0.069603  0.136656  0.101857   
3         0.084238  0.132769  1.000000  0.064823  0.151907  0.063715   
4         0.066051  0.069603  0.064823  1.000000  0.177653  0.048954   
5         0.105487  0.136656  0.151907  0.177653  1.000000  0.058376   
...            ...       ...       ...       ...       ...       ...   
3948      0.073159  0.079431  0.062382  0.053600  0.060514  0.060149   
3949      0.025635  0.068494  0.047559  0.079739  0.059630  0.076295   
3950      0.036152  0.036710  0.024081  0.002230  0.043309  0.071003   
3951     -0.008787 -0.007870 -0.014538  0.004015 -0.011805  0.007404   
3952      0.025334  0.040703  0.028089  0.040878  0.039363  0.084861   

movie_id      7         8         9         10    ...      3943

In [ ]:
item_predicted_ratings = np.dot(item_user_matrix, item_similarity)
item_predicted_ratings

array([[22.88784851, 18.43042017,  9.85968836, ...,  4.21363839,
         1.37347247,  7.07407829],
       [32.66717095, 32.03725942, 20.99772571, ..., 14.04260297,
         1.75746313, 19.56137862],
       [20.4492736 , 17.95348174, 10.99815199, ...,  4.75908058,
        -0.10929428,  6.22620825],
       ...,
       [ 6.53420891,  3.94970861,  2.78733052, ...,  2.03781022,
         1.2070057 ,  3.39430919],
       [32.53558681, 28.01000658, 17.08050477, ..., 10.34155713,
         4.51561997, 14.12146813],
       [76.56505336, 62.10926512, 38.97574875, ..., 34.26795642,
        13.64107169, 48.64727383]])

In [ ]:
item_predicted_ratings.shape


(6040, 3656)

In [ ]:
# make a copy of train and test datasets
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['ratings'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['ratings'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# The movies not rated by user is marked as 1 for prediction
dummy_train = dummy_train.pivot(index = 'user_id', columns = 'movie_id', values = 'ratings').fillna(1)

# The movies not rated by user is marked as 0 for evaluation
dummy_test = dummy_test.pivot(index ='user_id', columns = 'movie_id', values = 'ratings').fillna(0)

In [ ]:
dummy_train.shape

(6040, 3656)

In [ ]:
# np.multiply for cell-by-cell multiplication

item_final_ratings = np.multiply(item_predicted_ratings.T, dummy_train.T)
item_final_ratings.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,22.887849,32.667171,20.449274,6.774542,32.020872,17.642550,10.949038,35.568288,141.958151,676.394116,...,16.106556,23.433539,18.221169,3.626234,56.361474,158.507959,43.869084,6.534209,32.535587,306.260213
2,18.430420,32.037259,17.953482,6.755706,27.370317,18.065875,11.557988,35.678383,25.154147,562.891784,...,15.107779,21.830303,18.868467,3.164751,63.332086,165.563234,37.103196,3.949709,28.010007,62.109265
3,9.859688,20.997726,10.998152,2.965204,17.903841,12.723218,6.908999,26.111455,18.224864,86.294760,...,9.616137,12.250799,11.531785,2.064110,45.560926,104.123738,21.235990,2.787331,17.080505,38.975749
4,7.619924,17.497706,6.713599,1.528645,22.937642,10.362711,3.896316,28.009824,15.924845,67.528647,...,8.708438,9.293686,7.119704,1.559206,38.668362,331.609174,18.484674,2.344380,12.718009,37.596185
5,11.224808,20.151424,11.230180,2.696769,18.736529,13.817419,6.226978,25.500597,17.582027,95.532308,...,10.711382,10.830816,10.520103,2.082063,52.038328,104.544821,19.209990,2.824957,17.153470,37.002566


In [ ]:
item_final_ratings.iloc[42].sort_values(ascending = False)[0:5]

user_id
1680    1149.174350
1285    1114.552007
3224    1093.947090
4169    1068.807132
889      989.498446
Name: 43, dtype: float64

In [ ]:
test_item_features = X_test.pivot(index = 'movie_id', columns = 'user_id', values = 'ratings').fillna(0)
cosine_similarity_matrix = np.dot(test_item_features.T, test_item_features) / (np.linalg.norm(test_item_features, axis=0)[:, None] * np.linalg.norm(test_item_features, axis=0))


In [ ]:
pearson_similarity_matrix = test_item_features.corr(method='pearson')

In [ ]:

test_item_similarity = pearson_similarity_matrix
test_item_similarity[np.isnan(test_item_similarity)] = 0

print(test_item_similarity)
print("- "*10)
print(test_item_similarity.shape)

user_id      1         2         3         4         5         6         7     \
user_id                                                                         
1        1.000000 -0.006686 -0.004145 -0.002455  0.045419  0.114833  0.097867   
2       -0.006686  1.000000  0.048065  0.147345  0.093673  0.036327  0.030737   
3       -0.004145  0.048065  1.000000  0.155283 -0.008132  0.046935 -0.003277   
4       -0.002455  0.147345  0.155283  1.000000 -0.004817 -0.003007 -0.001941   
5        0.045419  0.093673 -0.008132 -0.004817  1.000000 -0.010614  0.127181   
...           ...       ...       ...       ...       ...       ...       ...   
6036     0.040430  0.029990  0.013801  0.016322  0.074755 -0.012035  0.047228   
6037     0.015439  0.077029  0.017899 -0.004673  0.005794  0.046039  0.076456   
6038    -0.002318  0.128304 -0.002174 -0.001288 -0.004547  0.028704 -0.001832   
6039    -0.006401 -0.009689  0.037168 -0.003558 -0.012560  0.018688  0.069476   
6040     0.053846 -0.002159 

In [ ]:
item_predicted_ratings_test = np.dot(test_item_features, test_item_similarity )
item_predicted_ratings_test

array([[158.72791981,  88.86351359,  64.8005319 , ...,  23.31620871,
         60.79891376,  85.32828504],
       [ 19.38133653,  34.54642372,  24.79851028, ...,   5.84412254,
         12.24599343,  23.94410803],
       [ 12.56514841,  23.25722087,  14.05064133, ...,   5.18720605,
          9.03408936,  13.35534801],
       ...,
       [  1.38021357,   1.70519123,   1.07359289, ...,   0.60224245,
          1.80029615,   2.54642346],
       [  0.7108292 ,   0.8182571 ,   0.78902211, ...,  -0.22404952,
          0.95777052,   1.13856274],
       [  9.36860088,  16.41601115,   8.57483257, ...,   2.72808444,
          7.06780759,  14.77214038]])

In [ ]:
test_item_final_rating = np.multiply(item_predicted_ratings_test.T, dummy_test)
test_item_final_rating.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,793.639599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratings_matrix['ratings'].describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: ratings, dtype: float64

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = test_item_final_rating.copy()
X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

print(pred)

[[3.61955012        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]]


In [ ]:

# total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

300063

In [ ]:
test = X_test.pivot(index = 'user_id', columns = 'movie_id', values = 'ratings')
test.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:


# RMSE Score

diff_sqr_matrix = (test - pred)**2
sum_of_squares_err = diff_sqr_matrix.sum().sum() # df.sum().sum() by default ignores null values

rmse = np.sqrt(sum_of_squares_err/total_non_nan)
print(rmse)



1.0929172572615944


In [ ]:
# Mean abslute error

mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

0.9676777408279681


In [111]:

# Function to generate recommendations for a target user
def generate_recommendations(user_id, k=5):
    # Get positively rated items by the target user
    positively_rated_items = X_train[(X_train['user_id'] == user_id) & (ratings_matrix['ratings'] >= 4)]

    # Find similar items to positively rated items
    similar_items = {}
    for item_id in positively_rated_items['movie_id']:
        similar_items[item_id] = item_similarity[item_id].nlargest(k+1).index.tolist()  # Exclude the item itself

    # Remove items already rated by the user
    for item_id, similar_list in similar_items.items():
        similar_items[item_id] = [item for item in similar_list if item not in positively_rated_items['movie_id'].values]

    # Generate recommendations
    recommendations = []
    for item_id, similar_list in similar_items.items():
        recommendations.extend(similar_list)

    return recommendations[:5]  # Return top 5 recommendations


In [106]:
user_id_test=1
recommendations = generate_recommendations(user_id_test)
print(recommendations)

[364, 594, 1, 1022, 1566]


<ipython-input-98-d554917b14ba>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positively_rated_items = X_train[(X_train['user_id'] == user_id) & (ratings_matrix['ratings'] >= 4)]


In [112]:
# Function to predict ratings for recommendations
def predict_ratings(user_id, recommendations):
    predicted_ratings = []
    for item_id in recommendations:
        # Find similar items to the target item
        similar_items = item_similarity[item_id]

        # Find users who have rated the target item
        users_who_rated_item = ratings_matrix[ratings_matrix['movie_id'] == item_id]

        # Calculate weighted average of ratings from similar items
        weighted_sum = 0
        similarity_sum = 0
        for similar_item_id, similarity_score in similar_items.iteritems():
            # Exclude the target item itself
            if similar_item_id == item_id:
                continue

            # Find users who have rated the similar item
            users_who_rated_similar_item = ratings_matrix[ratings_matrix['movie_id'] == similar_item_id]

            # Calculate intersection of users who rated both items
            common_users = set(users_who_rated_item['user_id']).intersection(set(users_who_rated_similar_item['user_id']))

            # Calculate weighted sum of ratings
            for user in common_users:
                user_rating = users_who_rated_similar_item[users_who_rated_similar_item['user_id'] == user]['ratings'].values[0]
                weighted_sum += similarity_score * user_rating
                similarity_sum += similarity_score

        # Avoid division by zero
        if similarity_sum != 0:
            predicted_rating = weighted_sum / similarity_sum
            predicted_ratings.append((item_id, predicted_rating))

    # Sort predicted ratings in descending order
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)

    return predicted_ratings[:5]  # Return top 1 predicted ratings


In [113]:
predictions=predict_ratings(user_id_test, recommendations)

<ipython-input-112-eda7372b63a2>:14: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for similar_item_id, similarity_score in similar_items.iteritems():


In [109]:
print(predictions)

[(1, 3.545739278108502)]
